## Import Library & Modules

In [ ]:
import pandas as pd
import numpy as np

from tff_model import run_tff_model

## Load Excel Data

In [13]:
df = pd.read_excel("system_inputs.xlsm", sheet_name="Inputs")
df

,C_s0,C_d0,V_r0,R_f0,Q_f,TMP,beta,V_s,V_d,k_d,k_f,t_start,t_end,dt
0,1,1,0.6,0,0.0001,5000000,0.0,0.5,0.1,0.1,0.00001,0,10000,20
1,1,1,0.6,0,0.0001,5000000,0.1,0.5,0.1,0.1,0.00001,0,10000,20
2,1,1,0.6,0,0.0001,5000000,0.2,0.5,0.1,0.1,0.00001,0,10000,20
3,1,1,0.6,0,0.0001,10000000,0.1,0.5,0.1,0.1,0.00001,0,10000,20
4,1,1,0.6,0,0.0001,10000000,0.1,0.5,0.1,0.1,0.00001,0,10000,20


In [ ]:
# get results for all runs
all_results = [] 

for run_id, row in df.iterrows():
    # initial conditions for each run
    y0 = [
        row["C_s0"],
        row["C_d0"],
        row["V_r0"],
        row["R_f0"]
    ]

    # inputs from excel for each run
    excel_inputs = {
        "y0": y0,
        "Q_f": row["Q_f"],
        "TMP": row["TMP"],
        "V_s": row["V_s"],
        "k_d": row["k_d"],
        "beta": row["beta"],
        "alpha": row["V_d"] / (row["V_s"] + row["V_d"])
    }

    # time settings for each run
    t_start = row["t_start"]
    t_end   = row["t_end"]
    dt      = row["dt"]

    t_eval = np.arange(t_start, t_end + dt, dt)

    # run the model for this set of inputs
    solution = run_tff_model(
        excel_inputs=excel_inputs,
        t_span=(t_start, t_end),
        t_eval=t_eval
    )

    # process results
    C_s, C_d, V_r, R_f = solution.y
    alpha = excel_inputs["alpha"]
    C_r = (1 - alpha) * C_s + alpha * C_d

    # store results in a DataFrame
    result_df = pd.DataFrame({
        "run_id": run_id + 1,
        "time": solution.t,
        "C_s": C_s,
        "C_d": C_d,
        "C_r": C_r,
        "V_r": V_r,
        "R_f": R_f
    })

    all_results.append(result_df)

In [15]:
df_outputs = pd.concat(all_results, ignore_index=True)
df_outputs

,run_id,time,C_s,C_d,C_r,V_r,R_f
0,1,0.0,1.0,1.0,1.0,0.6,0.0
1,1,20.0,1.0,1.0,1.0,0.6,0.0
2,1,40.0,1.0,1.0,1.0,0.6,0.0
3,1,60.0,1.0,1.0,1.0,0.6,0.0
4,1,80.0,1.0,1.0,1.0,0.6,0.0
...,...,...,...,...,...,...,...
2500,5,9920.0,1.0,1.0,1.0,0.6,0.0
2501,5,9940.0,1.0,1.0,1.0,0.6,0.0
2502,5,9960.0,1.0,1.0,1.0,0.6,0.0
2503,5,9980.0,1.0,1.0,1.0,0.6,0.0
